<a href="https://colab.research.google.com/github/CGnal/agos-ai-course/blob/day5/day5/03_Graph_Regularization_Graph_Neural_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab\"/></a>

In [1]:
!pip install \
    ipywidgets==7.6.5 \
    networkx==2.4 \
    matplotlib==3.2.2 \
    scikit-learn==0.24.0 \
    pandas==1.1.3 \
    numpy==1.19.2 \
    tensorflow==2.4.1 \
    neural-structured-learning==1.3.1 \
    stellargraph==1.2.1 

# Neural Graph Learning and Graph Regularization

In this tutorial, we will be creating a graph regularized version for a topic classification task. The task is to classify paper depending on their content. However in order to do so, we will also use the information encoded in the citation network that relates documents among each other. Of course, we do know that this kind of information is indeed powerful as papers belonging to the same subject tend to reference each other.   

### Load Dataset

For this tutorial we will be using the Cora dataset available in the stellargraph library 

In [2]:
from stellargraph import datasets

In [3]:
dataset = datasets.Cora()

In [4]:
%config Completer.use_jedi = False

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: Config option `use_jedi` not recognized by `IPCompleter`.
  """Entry point for launching an IPython kernel.


In [5]:
dataset.download()

In [6]:
label_index = {
      'Case_Based': 0,
      'Genetic_Algorithms': 1,
      'Neural_Networks': 2,
      'Probabilistic_Methods': 3,
      'Reinforcement_Learning': 4,
      'Rule_Learning': 5,
      'Theory': 6,
  }

In [7]:
G, labels = dataset.load()

We now create the Dataset object where we will both include information of the targeted sample (node) and its neighbors. In the following we will also allow to control the number of labelling instances to be used, in order to reproduce and evaluate the classification performance in a semi-supervised setting. 

In [8]:
import numpy as np
from sklearn import preprocessing, feature_extraction, model_selection

In [9]:
import tensorflow as tf
from tensorflow.train import Example, Features, Feature, Int64List, BytesList, FloatList

In [10]:
GRAPH_PREFIX="NL_nbr"

In [11]:
def _int64_feature(*value):
    """Returns int64 tf.train.Feature from a bool / enum / int / uint."""
    return Feature(int64_list=Int64List(value=list(value)))

def _bytes_feature(value):
    """Returns bytes tf.train.Feature from a string."""
    return Feature(
        bytes_list=BytesList(value=[value.encode('utf-8')])
    )

def _float_feature(*value):
    return Feature(float_list=FloatList(value=list(value)))


In [12]:
from functools import reduce
from typing import List, Tuple
import pandas as pd
import six

def addFeatures(x, y):
    res = Features()
    res.CopyFrom(x)
    res.MergeFrom(y)
    return res

def neighborFeatures(features: Features, weight: float, prefix: str):
    data = {f"{prefix}_weight": _float_feature(weight)}
    for name, feature in six.iteritems(features.feature):
        data[f"{prefix}_{name}"] = feature 
    return Features(feature=data)

def neighborsFeatures(neighbors: List[Tuple[Features, float]]):
    return reduce(
        addFeatures, 
        [neighborFeatures(sample, weight, f"{GRAPH_PREFIX}_{ith}") for ith, (sample, weight) in enumerate(neighbors)],
        Features()
    )

def getNeighbors(idx, adjMatrix, topn=5):
    weights = adjMatrix.loc[idx]
    return weights[weights>0].sort_values(ascending=False).head(topn).to_dict()
    

def semisupervisedDataset(G, labels, ratio=0.2, topn=5):
    n = int(np.round(len(labels)*ratio))
    
    labelled, unlabelled = model_selection.train_test_split(
        labels, train_size=n, test_size=None, stratify=labels
    )
    
    adjMatrix = pd.DataFrame.sparse.from_spmatrix(G.to_adjacency_matrix(), index=G.nodes(), columns=G.nodes())
    
    features = pd.DataFrame(G.node_features(), index=G.nodes())
    
    dataset = {
        index: Features(feature = {
            #"id": _bytes_feature(str(index)), 
            "id": _int64_feature(index),
            "words": _float_feature(*[float(x) for x in features.loc[index].values]), 
            "label": _int64_feature(label_index[label])
        })
        for index, label in pd.concat([labelled, unlabelled]).items()
    }
    
    trainingSet = [
        Example(features=addFeatures(
            dataset[exampleId], 
            neighborsFeatures(
                [(dataset[nodeId], weight) for nodeId, weight in getNeighbors(exampleId, adjMatrix, topn).items()]
            )
        ))
        for exampleId in labelled.index
    ]
    
    testSet = [Example(features=dataset[exampleId]) for exampleId in unlabelled.index]

    serializer = lambda _list: [e.SerializeToString() for e in _list]
    
    return serializer(trainingSet), serializer(testSet)


We split the dataset into a training set and a test set

In [13]:
trainingSet, testSet = semisupervisedDataset(G, labels)

In [14]:
import tensorflow as tf

In [15]:
from tensorflow.data import Dataset

In [16]:
vocabularySize = 1433

In [17]:
neighbors=2
defaultWord = tf.constant(0, dtype=tf.float32, shape=[vocabularySize])

def parseExample(example, training=True):
    schema = {
        'words': tf.io.FixedLenFeature([vocabularySize], tf.float32, default_value=defaultWord),
        'label': tf.io.FixedLenFeature((), tf.int64, default_value=-1)
    }
    
    if training is True:
        for i in range(neighbors):
            name = f"{GRAPH_PREFIX}_{i}"
            schema[f"{name}_weight"] = tf.io.FixedLenFeature([1], tf.float32, default_value=[0.0])
            schema[f"{name}_words"] = tf.io.FixedLenFeature([vocabularySize], tf.float32, default_value=defaultWord)
    
    features = tf.io.parse_single_example(example, schema)
    
    label = features.pop("label")
    return features, label

In [18]:
def sampleGenerator(dataset):
    def wrapper():
        for example in dataset:
            yield example
    return wrapper
            
myTrain = Dataset \
    .from_generator(sampleGenerator(trainingSet), output_types=tf.string, output_shapes=()) \
    .map(lambda x: parseExample(x, True))

myTest = Dataset \
    .from_generator(sampleGenerator(testSet), output_types=tf.string, output_shapes=()) \
    .map(lambda x: parseExample(x, False))


In [19]:
for features, labels in myTrain.batch(10).take(1):
    print(features)
    print(labels)

{'NL_nbr_0_weight': <tf.Tensor: shape=(10, 1), dtype=float32, numpy=
array([[1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.]], dtype=float32)>, 'NL_nbr_0_words': <tf.Tensor: shape=(10, 1433), dtype=float32, numpy=
array([[0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)>, 'NL_nbr_1_weight': <tf.Tensor: shape=(10, 1), dtype=float32, numpy=
array([[1.],
       [0.],
       [1.],
       [1.],
       [1.],
       [0.],
       [1.],
       [1.],
       [1.],
       [1.]], dtype=float32)>, 'NL_nbr_1_words': <tf.Tensor: shape=(10, 1433), dtype=float32, numpy=
array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
     

In [20]:
for features, labels in myTest.batch(10).take(1):
    print(features)
    print(labels)

{'words': <tf.Tensor: shape=(10, 1433), dtype=float32, numpy=
array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)>}
tf.Tensor([6 1 2 2 0 0 3 3 0 6], shape=(10,), dtype=int64)


### Creating the model 

We now create the model that we will use to classify the documents

In [21]:
layers = [50, 50]

In [22]:
"""Creates a functional API-based multi-layer perceptron model."""
def create_model(num_units):
    inputs = tf.keras.Input(
          shape=(vocabularySize,), dtype='float32', name='words'
    )

    # outputs = tf.keras.layers.Dense(len(label_index), activation='softmax')(inputs)

    cur_layer =  inputs

    for num_units in layers:
        cur_layer = tf.keras.layers.Dense(num_units, activation='relu')(cur_layer)
        cur_layer = tf.keras.layers.Dropout(0.8)(cur_layer)

    outputs = tf.keras.layers.Dense(len(label_index), activation='softmax')(cur_layer)

    return tf.keras.Model(inputs, outputs=outputs)

In [23]:
from tensorflow.keras.callbacks import TensorBoard

#### Vanilla Model

We first train a simple, vanilla version that does not use the citation network information

In [24]:
model = create_model([50, 50])

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy'])

In [25]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
words (InputLayer)           [(None, 1433)]            0         
_________________________________________________________________
dense (Dense)                (None, 50)                71700     
_________________________________________________________________
dropout (Dropout)            (None, 50)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 50)                2550      
_________________________________________________________________
dropout_1 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 7)                 357       
Total params: 74,607
Trainable params: 74,607
Non-trainable params: 0
_________________________________________________________

In [26]:
model.fit(myTrain.batch(128), epochs=200, verbose=1, validation_data=myTest.batch(128),
          callbacks=[TensorBoard(log_dir='/tmp/noRegularization')])

Epoch 1/200


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/functional.py:595: UserWarning: Input dict contained keys ['NL_nbr_0_weight', 'NL_nbr_0_words', 'NL_nbr_1_weight', 'NL_nbr_1_words'] which did not match any model input. They will be ignored by the model.
  [n for n in tensors.keys() if n not in ref_input_names])


5/5 [==============================] - 2s 257ms/step - loss: 2.0266 - accuracy: 0.1286 - val_loss: 1.9498 - val_accuracy: 0.1191
Epoch 2/200
5/5 [==============================] - 1s 134ms/step - loss: 2.0461 - accuracy: 0.1541 - val_loss: 1.9392 - val_accuracy: 0.1745
Epoch 3/200
5/5 [==============================] - 1s 128ms/step - loss: 2.0161 - accuracy: 0.1524 - val_loss: 1.9306 - val_accuracy: 0.2350
Epoch 4/200
5/5 [==============================] - 1s 129ms/step - loss: 1.9935 - accuracy: 0.1825 - val_loss: 1.9235 - val_accuracy: 0.2756
Epoch 5/200
5/5 [==============================] - 1s 131ms/step - loss: 1.9533 - accuracy: 0.1813 - val_loss: 1.9176 - val_accuracy: 0.2959
Epoch 6/200
5/5 [==============================] - 1s 137ms/step - loss: 1.9619 - accuracy: 0.2012 - val_loss: 1.9126 - val_accuracy: 0.3024
Epoch 7/200
5/5 [==============================] - 1s 127ms/step - loss: 1.9285 - accuracy: 0.2066 - val_loss: 1.9077 - val_accuracy: 0.3121
Epoch 8/200
5/5 [========

#### Graph Regularized Version

We now create the graph-regularized version that uses the citation network information

In [27]:
base_model = create_model([50, 50])

In [28]:
import neural_structured_learning as nsl

In [29]:
graph_reg_config = nsl.configs.make_graph_reg_config(
    max_neighbors=2,
    multiplier=0.1,
    distance_type=nsl.configs.DistanceType.L2,
    sum_over_axis=-1)
graph_reg_model = nsl.keras.GraphRegularization(base_model,
                                                graph_reg_config)
graph_reg_model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy'])
#graph_reg_model.fit(train_dataset, epochs=200, verbose=1)

In [30]:
graph_reg_model.fit(myTrain.batch(128), epochs=200, verbose=1, validation_data=myTest.batch(128),
          callbacks=[TensorBoard(log_dir='/tmp/regularization')])

Epoch 1/200


/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/indexed_slices.py:437: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/GraphRegularization/graph_loss/Reshape_1:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/GraphRegularization/graph_loss/Reshape:0", shape=(None, 7), dtype=float32), dense_shape=Tensor("gradient_tape/GraphRegularization/graph_loss/Cast:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "shape. This may consume a large amount of memory." % value)


5/5 [==============================] - 3s 201ms/step - loss: 2.0773 - accuracy: 0.1336 - scaled_graph_loss: 0.0062 - val_loss: 1.9514 - val_accuracy: 0.1219
Epoch 2/200
5/5 [==============================] - 1s 125ms/step - loss: 2.0178 - accuracy: 0.1899 - scaled_graph_loss: 0.0056 - val_loss: 1.9369 - val_accuracy: 0.1994
Epoch 3/200
5/5 [==============================] - 1s 127ms/step - loss: 1.9884 - accuracy: 0.1983 - scaled_graph_loss: 0.0045 - val_loss: 1.9263 - val_accuracy: 0.2608
Epoch 4/200
5/5 [==============================] - 1s 123ms/step - loss: 1.9576 - accuracy: 0.1527 - scaled_graph_loss: 0.0038 - val_loss: 1.9179 - val_accuracy: 0.2982
Epoch 5/200
5/5 [==============================] - 1s 129ms/step - loss: 1.9633 - accuracy: 0.1822 - scaled_graph_loss: 0.0034 - val_loss: 1.9108 - val_accuracy: 0.3139
Epoch 6/200
5/5 [==============================] - 1s 127ms/step - loss: 1.9133 - accuracy: 0.2266 - scaled_graph_loss: 0.0030 - val_loss: 1.9044 - val_accuracy: 0.312